In [39]:
# importando as libs
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [40]:
# light gbm como classificador

In [41]:
# importando o seaborn somente para usar umas bases de dados IRIS
import seaborn as sns

In [42]:
df_iris = sns.load_dataset('iris')
df_iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [43]:
# separando X e Y
X_iris = df_iris.drop(['species'], axis=1)
y_iris = df_iris['species']

In [44]:
# quebrando a base inteira em parcelas de treino e validacao
X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(X_iris, y_iris, random_state=42)

In [45]:
# lgbm como classificador

classificador_lgbm = lgb.LGBMClassifier()

In [46]:
# pra testar a acuracia do modelo, vamos realizar uma validacao cruzada
from sklearn.model_selection import cross_val_score


acuracia = 100 * cross_val_score(classificador_lgbm, X_iris_train, y_iris_train).mean()
print('acurácia: ', acuracia)

acurácia:  93.71541501976284


In [47]:
# lightgbm permite tunning de hiperparametros
# aqui por ex, vamos tentar tunar a maxima profundidade da arvore

classificador_lgbm_tunado = lgb.LGBMClassifier(max_depth=2)
acuracia_tunada = 100 * cross_val_score(classificador_lgbm_tunado, X_iris_train, y_iris_train).mean()
print('acurácia tunada: ', acuracia_tunada)

acurácia tunada:  93.71541501976284


In [48]:
# lightgbm permite tunning de hiperparametros
# aqui por ex, vamos trocar o boosting convencional por dart

classificador_lgbm_dart = lgb.LGBMClassifier(boosting_type='dart')
acuracia_tunada_dart = 100 * cross_val_score(classificador_lgbm_dart, X_iris_train, y_iris_train).mean()
print('acurácia tunada dart: ', acuracia_tunada_dart)

acurácia tunada dart:  93.71541501976284


In [49]:
## testamos tudo e definimos entao qual a melhor config possivel
# podemos entao utiliza-lo permanentemente daqui pra frente

classificador_final = lgb.LGBMClassifier()
classificador_final.fit(X_iris_train, y_iris_train)

LGBMClassifier()

In [50]:
# modelo ajustado, podemos agora fazer previsoes

predicoes_iris = classificador_final.predict(X_iris_test)
predicoes_iris[0:5] #mostrando so os 5 primeiros valores preditos

array(['versicolor', 'setosa', 'virginica', 'versicolor', 'versicolor'],
      dtype=object)

In [51]:
# como era na realidade os 5 primeiros preditos?
y_iris_test[0:5]

73     versicolor
18         setosa
118     virginica
78     versicolor
76     versicolor
Name: species, dtype: object

In [52]:
## parece que acertou bem, mas temos que verificar todos

acertos = (predicoes_iris == y_iris_test).sum()
total_testes = len(y_iris_test)

acuracia = 100*acertos/total_testes

print('percentual da taxa de acerto: ', acuracia)

percentual da taxa de acerto:  94.73684210526316


In [53]:
###################

In [54]:
# light gbm como regressor

In [55]:
## dados de milhas por galao
df_mpg = sns.load_dataset('mpg')
df_mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [56]:
# separando X e Y
X_mpg = df_mpg.drop(['mpg', 'origin', 'name'], axis=1)
y_mpg = df_mpg['mpg']

In [57]:
# quebrando a base inteira em parcelas de treino e validacao
X_mpg_train, X_mpg_test, y_mpg_train, y_mpg_test = train_test_split(X_mpg, y_mpg, random_state=42)

In [58]:
## testando modelos e tunagens por validacao cruzada

In [59]:
# regressor lgbm puro
regressor_lgbm = lgb.LGBMRegressor()
cross_val_score(regressor_lgbm, X_mpg_train, y_mpg_train, scoring='neg_root_mean_squared_error').mean()

-3.0602968789895

In [60]:
# regressor lgbm com dart

regressor_lgbm_dart = lgb.LGBMRegressor(boosting_type='dart')

cross_val_score(regressor_lgbm_dart, X_mpg_train, y_mpg_train, scoring='neg_root_mean_squared_error').mean()

-3.8958190536698334

In [61]:
# regressor lgbm com outra profundidade

regressor_lgbm_tunado = lgb.LGBMRegressor(max_depth=2)

cross_val_score(regressor_lgbm_tunado, X_mpg_train, y_mpg_train, scoring='neg_root_mean_squared_error').mean()

-3.085218235455716

In [62]:
## menor erro foi lgbm puro
# ele será o campeao e será utilizado daqui em diante
# podemos ajusta-lo (fit) com todos os dados de treino agora

regressor_campeao = lgb.LGBMRegressor()
regressor_campeao.fit(X_mpg_train, y_mpg_train, eval_metric='neg_root_mean_squared_error')

LGBMRegressor()

In [63]:
# predizendo valores com a parcela de teste
predicoes_mpg = regressor_campeao.predict(X_mpg_test)

#olhando os 5 primeiros previstos
predicoes_mpg[0:5]

array([32.62089047, 30.47819635, 21.1453571 , 15.63407173, 13.12103143])

In [64]:
# observando os valores reais dos 5 primeiros 
y_mpg_test[0:5]

198    33.0
396    28.0
33     19.0
208    13.0
93     14.0
Name: mpg, dtype: float64

In [65]:
## erro final desse teste

from sklearn.metrics import mean_squared_error
import math

mse = mean_squared_error(y_mpg_test, predicoes_mpg)
rmse = math.sqrt(mse)

print('erro: ',rmse)

erro:  2.526606943873821
